In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key
)

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is 72 degrees and Sunny."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat([weather_agent], termination_condition=text_termination)

async def run_team() -> None:
    result = await single_agent_team.run(task="What is the weather in New York?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()


/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:981: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 8, 54, 90479, tzinfo=datetime.timezone.utc), content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 8, 55, 105643, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_f4Ieq31ggId4IgkXRlk6J6zN', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 8, 55, 106734, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call_f4Ieq31ggId4IgkXRlk6J6zN', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_u

In [3]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="What is the weather in New York?"):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()



source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 20, 7, 8, 55, 561906, tzinfo=datetime.timezone.utc) content='What is the weather in New York?' type='TextMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=136, completion_tokens=16) metadata={} created_at=datetime.datetime(2025, 6, 20, 7, 8, 57, 580243, tzinfo=datetime.timezone.utc) content='The weather in New York is 72 degrees and sunny. TERMINATE' type='TextMessage'
Stop Reason: Text 'TERMINATE' mentioned


In [4]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
What is the weather in Seattle?
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='call_lLkrAnfS51rLbirL4CPweUfx', arguments='{"city":"Seattle"}', name='get_weather')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', call_id='call_lLkrAnfS51rLbirL4CPweUfx', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
The weather in Seattle is 72 degrees and Sunny.
---------- TextMessage (assistant) ----------
The weather in Seattle is 72 degrees and sunny. TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 8, 57, 589592, tzinfo=datetime.timezone.utc), content='What is the weather in Seattle?', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 8, 58, 278845, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_lLkrAnfS51rLbirL4CPweUfx', arguments='{"city":"Seattle"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 8, 58, 281537, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', call_id='call_lLkrAnfS51rLbirL4CPweUfx', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant',

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=termination)

In [6]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="Write a short poem about fall season.")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a short poem about fall season.
---------- TextMessage (primary) ----------
Golden leaves drift to the ground,  
A crispness lingers all around.  
Amber skies and cooler air,  
Pumpkin spice is everywhere.  

Whispers weave through forests deep,  
Nature slows, preparing sleep.  
Scarlet, ochre, rust, and gold,  
Fall's fleeting beauty never grows old.  
---------- TextMessage (critic) ----------
What a lovely poem capturing the essence of fall! The imagery is vivid and evokes the beauty and comfort of the season. However, here are a few suggestions for refinement:

1. You could introduce a bit more sensory detail to deepen the reader's immersion, such as the sound of leaves crunching underfoot or the scent of woodsmoke in the air.  
2. While "Pumpkin spice is everywhere" is relatable, it feels slightly modern compared to the otherwise timeless tone. Perhaps consider rephrasing it to blend better with the lyrical quality.  
3. The last lin

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 8, 59, 97822, tzinfo=datetime.timezone.utc), content='Write a short poem about fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=68), metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 9, 0, 310567, tzinfo=datetime.timezone.utc), content="Golden leaves drift to the ground,  \nA crispness lingers all around.  \nAmber skies and cooler air,  \nPumpkin spice is everywhere.  \n\nWhispers weave through forests deep,  \nNature slows, preparing sleep.  \nScarlet, ochre, rust, and gold,  \nFall's fleeting beauty never grows old.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=112, completion_tokens=162), metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 9, 2, 885334, tzinfo=datetime.timezone.utc), content='What a lovely poem capturing 

In [7]:
await Console(reflection_team.run_stream(task="중국 당나라 시풍으로 시를 써보세요."))

---------- TextMessage (user) ----------
중국 당나라 시풍으로 시를 써보세요.
---------- TextMessage (primary) ----------
물론입니다! 당나라 시풍은 간결하면서도 자연과 인간의 조화를 묘사하며, 깊은 철학적 정취가 담긴 것이 특징입니다. 아래는 그런 분위기를 담아본 시입니다:

---

**산거(山居)**  
고목에 새 울고, 연무는 산을 감도네.  
밤 이슬 내려앉으니 달빛 흰 바위 적시고,  
물소리 먼데 흐르고, 바람에 솔 냄새 깃드네.  
인생이란 꿈같은 것, 이 순간을 누가 쥘쏘냐.  

---

이 시는 자연 속에서의 고요한 삶과 인간의 유한함을 사유하는 내용을 담았습니다. 당나라 시풍의 요소를 살려보았는데, 마음에 드셨기를 바랍니다!
---------- TextMessage (critic) ----------
시의 분위기가 당나라 특유의 고요함과 철학적 정취를 잘 담았습니다. 특히 자연과 인간의 연결을 묘사한 부분이 깊은 울림을 줍니다. 하지만 다음과 같은 제안을 드립니다:

1. 마지막 행에서 보다 복합적이고 함축적인 표현을 사용하면 한층 더 깊은 여운을 줄 수 있을 것 같습니다. 예를 들어 "이 순간을 누가 쥘쏘냐" 대신 "인연조차 바람 같아라" 같은 표현은 어떨까요?

2. '물소리 먼데 흐르고' 부분에 약간 더 구체적인 묘사를 추가하면 독자가 자연 속 장면을 더욱 선명히 그릴 수 있을 것 같습니다.

수정된 버전을 기대하겠습니다!
---------- TextMessage (primary) ----------
귀한 비평에 깊이 감사드립니다. 말씀하신 제안을 바탕으로 보다 함축적이고 생동감 있는 표현을 더해보았습니다. 다음은 수정된 버전입니다.

---

**산거(山居)**  
고목에 새 울고, 연무는 산을 감도네.  
밤 이슬 곱게 내려 첨첨히 달빛 어린 바위,  
먼 물결 계곡 흐르고 솔 향은 구름에 깃드네.  
인생 유한하니, 인연도 바람 같아라.  

---

마지막 행을

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 9, 6, 42793, tzinfo=datetime.timezone.utc), content='중국 당나라 시풍으로 시를 써보세요.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=458, completion_tokens=185), metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 9, 8, 200323, tzinfo=datetime.timezone.utc), content='물론입니다! 당나라 시풍은 간결하면서도 자연과 인간의 조화를 묘사하며, 깊은 철학적 정취가 담긴 것이 특징입니다. 아래는 그런 분위기를 담아본 시입니다:\n\n---\n\n**산거(山居)**  \n고목에 새 울고, 연무는 산을 감도네.  \n밤 이슬 내려앉으니 달빛 흰 바위 적시고,  \n물소리 먼데 흐르고, 바람에 솔 냄새 깃드네.  \n인생이란 꿈같은 것, 이 순간을 누가 쥘쏘냐.  \n\n---\n\n이 시는 자연 속에서의 고요한 삶과 인간의 유한함을 사유하는 내용을 담았습니다. 당나라 시풍의 요소를 살려보았는데, 마음에 드셨기를 바랍니다!', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=660, completion_tokens=173), metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 9, 10, 367762, tzinfo=datetime.timezone.utc), content='시의 분위기가 당나라 특유의 고요함과 철학적

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=termination)

In [9]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="What is the weather in New York?")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="서울의 기후는 어때?"))

---------- TextMessage (user) ----------
서울의 기후는 어때?
---------- TextMessage (lazy_assistant) ----------
서울의 기후는 **온대 계절풍 기후**(중위도 대륙성 기후, Köppen의 기후 분류 기준에서는 Dwa 또는 Cwa)로 분류되며, 뚜렷한 사계절을 가지고 있습니다. 주요 특징은 다음과 같습니다:

### 봄 (3월 ~ 5월)
- 날씨: 대체로 맑고 건조, 그러나 미세먼지나 황사가 발생할 가능성이 있음.
- 온도: 비교적 따뜻하며 10°C에서 20°C 사이.

### 여름 (6월 ~ 8월)
- 날씨: 덥고 습하며 장마철에 많은 비가 내림.
- 온도: 25°C에서 35°C까지 상승하며, 습도가 매우 높음.
- 특징: 7월과 8월은 서울에서 가장 더운 시기.

### 가을 (9월 ~ 11월)
- 날씨: 쾌적하고 맑은 날씨가 많음.
- 온도: 기온은 10°C에서 20°C 사이로 선선함.
- 특징: 청명한 하늘과 단풍이 아름다운 시기.

### 겨울 (12월 ~ 2월)
- 날씨: 춥고 건조하며 가끔 눈이 내림.
- 온도: -10°C에서 5°C로 추운 날씨가 이어짐.
- 특징: 난방이 필수인 계절이며, 한파가 찾아올 때는 매우 추워짐.

서울은 여름과 겨울의 기온 차이가 큰 편이고, 여름에는 습도가 높아 무더위를 느낄 수 있으며, 겨울에는 찬 바람이 강합니다.
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_ZZLHbcBGZLvLXmGEABKHp0yU', arguments='{}', name='transfer_to_user')]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 9, 13, 517850, tzinfo=datetime.timezone.utc), content='서울의 기후는 어때?', type='TextMessage'), TextMessage(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=68, completion_tokens=379), metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 9, 17, 785607, tzinfo=datetime.timezone.utc), content='서울의 기후는 **온대 계절풍 기후**(중위도 대륙성 기후, Köppen의 기후 분류 기준에서는 Dwa 또는 Cwa)로 분류되며, 뚜렷한 사계절을 가지고 있습니다. 주요 특징은 다음과 같습니다:\n\n### 봄 (3월 ~ 5월)\n- 날씨: 대체로 맑고 건조, 그러나 미세먼지나 황사가 발생할 가능성이 있음.\n- 온도: 비교적 따뜻하며 10°C에서 20°C 사이.\n\n### 여름 (6월 ~ 8월)\n- 날씨: 덥고 습하며 장마철에 많은 비가 내림.\n- 온도: 25°C에서 35°C까지 상승하며, 습도가 매우 높음.\n- 특징: 7월과 8월은 서울에서 가장 더운 시기.\n\n### 가을 (9월 ~ 11월)\n- 날씨: 쾌적하고 맑은 날씨가 많음.\n- 온도: 기온은 10°C에서 20°C 사이로 선선함.\n- 특징: 청명한 하늘과 단풍이 아름다운 시기.\n\n### 겨울 (12월 ~ 2월)\n- 날씨: 춥고 건조하며 가끔 눈이 내림.\n- 온도: -10°C에서 5°C로 추운 날씨가 이어짐.\n- 특징: 난방이 필수인 계절이며, 한파가 찾아올 때는 매우 추워짐.\n\n서울은 여름과 겨울의 기온 차이가 큰